In [25]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy

import torch.distributions as tdist

from ipywidgets import interact

# Experiment 1: Learning 2D standard form posterior

We are assuming that our posterior is of the form 
$$
p(\xi_1, \xi_2) \propto \xi_1^{h_1} \xi_2^{h_2} e^{-n \xi_1^{2k_1} \xi_2^{2k_2}}, \quad (\xi_1, \xi_2) \in [0, 1]^2
$$
The posterior contours would look like the following:

In [41]:
N = 200
xx, yy = np.meshgrid(np.linspace(0, 1, num=50), np.linspace(0, 1, num=N))


@interact(
    n=(1, 10000),
    k1=(0, 5),
    k2=(0, 5),
    h1=(0, 5),
    h2=(0, 5)
)
def _plot_contour(n=5, k1=2, k2=1, h1=1, h2=1):
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    z = (xx ** h1) * (yy ** h2) * np.exp(-n * (xx **(2 * k1)) * (yy **(2*k2)))
    normalising_const = np.sum(z) / (N**2)
    
    lambdas = np.array([(h1 + 1) / (2 * k1), (h2 + 1) / (2 * k2)])
    rlct = np.min(lambdas)
    m = np.sum(lambdas == rlct)
    print(
        f"-logZ = {-np.log(normalising_const)}\n"
        f"RLCT= {rlct}\n"
        f"m={m}\n"
        f"rlct log(n) - (m - 1) log log n= {rlct * np.log(n) - (m -1) * np.log(np.log(n))}"
    )
    ax.contourf(xx, yy, z / normalising_const, levels=100)
    ax.set_xlabel("$\\xi_1$")
    ax.set_ylabel("$\\xi_2$")

interactive(children=(IntSlider(value=5, description='n', max=10000, min=1), IntSlider(value=2, description='k…

Given a variational family $\mathcal{Q}$, we wish to find a variational approximation $\hat{q} \in \mathcal{Q}$ that minimises the KL-divergence $KL(\hat{q} || p(\xi_1, \xi_2)$. Notice though that this is equivalent to maximising the ELBO = Evidence Lower Bound. 

$$
\mathrm{ELBO(q)} = -\mathbb{E}_q[\log(q)] + \mathbb{E}_q[n\xi_1^{2k_1}\xi_2^{2k_2}] - \mathbb{E}_q[\log \xi_1^{h_1}\xi_2^{h_2}]
$$